In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 12, 3, padding=1)
        self.batch1 = nn.BatchNorm2d(num_features = 12)
        self.conv2 = nn.Conv2d(12, 16, 3, padding=1)
        self.batch2 = nn.BatchNorm2d(num_features = 16)
        self.drop1 = nn.Dropout2d(0.2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.batch3 = nn.BatchNorm2d(num_features = 32)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.batch4 = nn.BatchNorm2d(num_features = 32)
        self.drop2 = nn.Dropout2d(0.2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(32, 64, 3)
        self.batch5 = nn.BatchNorm2d(num_features = 64)
        self.conv6 = nn.Conv2d(64, 64, 3)
        self.conv7 = nn.Conv2d(64, 10, 3)

    def forward(self, x):
        x = self.batch1(F.relu(self.conv1(x)))
        x = self.batch2(F.relu(self.conv2(x)))
        x = self.drop1(self.pool1(x))
        x = self.batch3(F.relu(self.conv3(x)))
        x = self.batch4(F.relu(self.conv4(x)))
        x = self.drop2(self.pool2(x))
        x = self.batch5(F.relu(self.conv5(x)))
        x = F.relu(self.conv6(x))
        x = self.conv7(x)
        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             120
       BatchNorm2d-2           [-1, 12, 28, 28]              24
            Conv2d-3           [-1, 16, 28, 28]           1,744
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
         Dropout2d-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 32, 14, 14]           4,640
       BatchNorm2d-8           [-1, 32, 14, 14]              64
            Conv2d-9           [-1, 32, 14, 14]           9,248
      BatchNorm2d-10           [-1, 32, 14, 14]              64
        MaxPool2d-11             [-1, 32, 7, 7]               0
        Dropout2d-12             [-1, 32, 7, 7]               0
           Conv2d-13             [-1, 64, 5, 5]          18,496
      BatchNorm2d-14             [-1, 6

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
# I am turning OFF TQDM. I don't want progress bar for this project
# Please note that it will not affect your train/test accuracy
#from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    #pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Average loss: 0.0369, Accuracy: 9875/10000 (98.75%)


Test set: Average loss: 0.0292, Accuracy: 9907/10000 (99.07%)


Test set: Average loss: 0.0221, Accuracy: 9934/10000 (99.34%)


Test set: Average loss: 0.0193, Accuracy: 9928/10000 (99.28%)


Test set: Average loss: 0.0177, Accuracy: 9941/10000 (99.41%)


Test set: Average loss: 0.0202, Accuracy: 9930/10000 (99.30%)


Test set: Average loss: 0.0168, Accuracy: 9946/10000 (99.46%)


Test set: Average loss: 0.0168, Accuracy: 9953/10000 (99.53%)


Test set: Average loss: 0.0175, Accuracy: 9942/10000 (99.42%)


Test set: Average loss: 0.0206, Accuracy: 9937/10000 (99.37%)


Test set: Average loss: 0.0195, Accuracy: 9933/10000 (99.33%)


Test set: Average loss: 0.0190, Accuracy: 9938/10000 (99.38%)


Test set: Average loss: 0.0166, Accuracy: 9937/10000 (99.37%)


Test set: Average loss: 0.0173, Accuracy: 9943/10000 (99.43%)


Test set: Average loss: 0.0207, Accuracy: 9936/10000 (99.36%)


Test set: Average loss: 0.0170, Accurac